In [1]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
# 訓練データ
fashion_mnist_train = FashionMNIST("./FashionMNIST", train=True, download=True, transform=transforms.ToTensor())

# テストデータ
fashion_mnist_test = FashionMNIST("./FashionMNIST", train=False, download=True, transform=transforms.ToTensor())

batch_size = 128
train_loader = DataLoader(fashion_mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test, batch_size=batch_size, shuffle=True)

In [3]:
import torch
from torch import nn
from torch.autograd import Variable as V

class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)

In [4]:
conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

In [5]:
# 試しに畳み込みレイヤーに入力してみる
test_input = V(torch.ones(1, 1, 28, 28))
conv_output_size = conv_net(test_input).size()
print(conv_output_size)
conv_output_size = conv_output_size[-1]

torch.Size([1, 1024])


In [6]:
# ２層のmlp
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200, 10)
)

In [7]:
# ２層の畳み込みレイヤーと２層のmlpをつなげてCNNを構成
net = nn.Sequential(
    conv_net,
    mlp
)

In [8]:
def eval_net(net, data_loader):
    # ネットワークを評価モードにする(dropoutやバッチノーマリゼーションを無効化する)
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        x = V(x)
        y = V(y)
        # 確率が最大のクラスを取得
        _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    # ミニバッチごとの正解と予測結果を一つにまとめる
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 予測精度を計算
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.data[0]

In [11]:
from torch import optim
from matplotlib import pyplot as plt
from tqdm import tqdm


def train_net(net, train_loader, test_loader, optimizer_cls=optim.Adam, loss_fn=nn.CrossEntropyLoss(), n_iter=10):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # ネットワークを訓練モードにする(dropoutやバッチノーマリゼーションを有効化する)
        net.train()
        n = 0
        n_acc = 0
        for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = V(x)
            yy = V(y)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.data[0]
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().data[0]
        train_losses.append(running_loss / i)
        train_acc.append(n_acc / n)
        val_acc.append(eval_net(net, test_loader))
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)

In [12]:
train_result = train_net(net, train_loader, test_loader, n_iter=20)

  0%|          | 0/469 [00:00<?, ?it/s]


IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python number